#  Submit test

In [ ]:
%aiida
# General imports
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from ase import Atoms
from IPython.display import display, clear_output, HTML
import itertools

from traitlets import dlink, Dict

# AiiDA imports
#from aiida_cp2k.utils import Cp2kInput
from aiida.orm import SinglefileData
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget,  StructureBrowserWidget
from aiidalab_widgets_base.utils import string_range_to_set
from aiida_cp2k.workchains.base import Cp2kBaseWorkChain

# Custom imports

from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
#from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.inputs import InputDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.cp2k_input_validity import validate_input #input_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
#from apps.surfaces.widgets.get_cp2k_input import Get_CP2K_Input

from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
## STRUCTURE SELECTOR
structure_selector = StructureManagerWidget(importers=[
    ("AiiDA database", StructureBrowserWidget()),
    ("Import from computer", StructureUploadWidget()),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

In [ ]:
## DESCRIPTION OF CALCULATION
process_description = ipw.Text(description='Process description: ',
                               placeholder='Type the name here.',
                               style={'description_width': '120px'},
                               layout={'width': '70%'})

In [ ]:
# Code
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')

input_details = InputDetails()

create_input = ipw.Button(description="Create Input")

analyzer = StructureAnalyzer()

dlink((structure_selector, 'structure'), (analyzer, 'structure'))
dlink((analyzer, 'details'), (input_details, 'details'))
dlink((computer_code_dropdown, 'selected_code'),(input_details, 'selected_code'))

In [ ]:
def get_plain_input():
    error_msg,submit_dict = CP2K2DICT(input_lines = input_details.plain_input.value)
    if error_msg != "":
        print(error_msg)
    else:
        input_builder = Cp2kBaseWorkChain.get_builder()

        # code
        input_builder.cp2k.code = computer_code_dropdown.selected_code
        # structure
        input_builder.cp2k.file.input_xyz = make_geom_file(structure_selector.structure_node, Str("mol_on_slab.xyz")) 

        if details_dict['calc_type'] != 'Full DFT':
            # slab potential
            slab_element=list(slab_analyzed['slab_elements'])[0]
            input_builder.cp2k.file.pot_f = SinglefileData(file='/home/aiida/apps/surfaces/slab/'+slab_element+'.pot')
            mol_indexes = list(itertools.chain(*slab_analyzed['all_molecules']))
            input_builder.cp2k.file.mol_xyz = make_geom_file(structure_selector.structure_node,
                                                             Str("mol.xyz"),
                                                             selection=List(list=mol_indexes))        

        input_builder.handler_overrides = Dict(dict={'resubmit_unconverged_geometry':True})

        input_builder.cp2k.metadata = details_dict['metadata']
        input_builder.metadata.label = "SlabGeoOptWorkChain"
        input_builder.metadata.description = process_description.value

        input_builder.cp2k.metadata['label'] = "slab_geo_opt"
        input_builder.cp2k.metadata['description'] = process_description.value
        input_builder.cp2k.parameters = Dict(dict=submit_dict)
        #input_builder.cp2k.settings = Dict(dict={'additional_retrieve_list': ["runtime.callgraph"]})
    return input_builder

In [ ]:
btn_submit = SubmitButtonWidget(process=Cp2kBaseWorkChain, input_dictionary_function=get_plain_input)
btn_submit.btn_submit.disabled=True

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit.btn_submit.disabled=True    

def create(_=None):
    can_submit, msg, details_dict = input_details.return_final_dictionary()
    
    btn_submit.btn_submit.disabled = not can_submit
    with output:
        clear_output()
        if can_submit:
            print("input created")
        else:
            print(msg)
            
        

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
display(ipw.VBox([input_details,computer_code_dropdown,create_input]))

In [ ]:
## DISPLAY SUBMIT BUTTON
display(btn_submit, output)